# 本周学习总结

## 学习周期
**周次**：第4周  
**学习内容**：Introduction to Economic Modeling and Data Science
 -    DataFrames and Series in Pandas             

## DataFrames and Series in Pandas

### The Index
- **Index对象**：类似数组但不可变，支持集合运算
-  **`.reindex()`**  ：重新排列、插入或删除数据

```python
import pandas as pd

# 创建Series并查看index
data = pd.Series([1, 2, 3], index=['a', 'b', 'c'])
print(data.index)  # Index(['a', 'b', 'c'], dtype='object')

# reindex操作
new_index = ['c', 'b', 'a', 'd']
reindexed = data.reindex(new_index)
print(reindexed)
```
**MultiIndex（多级索引）**
- 在**DataFrame**行或列上创建层次化索引
- 支持更复杂的数据组织方式

```python
# 创建MultiIndex示例
multi_idx = pd.MultiIndex.from_tuples([
    ('A', 1), ('A', 2), ('B', 1), ('B', 2)
], names=['Letter', 'Number'])

df_multi = pd.DataFrame({'Values': [10, 20, 30, 40]}, index=multi_idx)
print(df_multi)

# MultiIndex索引
print(df_multi.loc['A'])        # 选择第一层索引
print(df_multi.loc[('A', 1)])   # 选择具体组合
```
 **Index性能**
- Index加速数据查找（基于哈希表）
- 无Index的DataFrame使用逐行扫描（慢）

### Storage Formats
#### **CSV文件**
-  `.read_csv()`  : 读取CSV
-  `.to_csv()`  : 写入CSV
-  **注意**  ：CSV不保存**index**和**data types**

```python
# 读取CSV
df = pd.read_csv('data.csv')

# 写入CSV（不保存index）
df.to_csv('output.csv', index=False)

# 读取时指定index列
df_indexed = pd.read_csv('data.csv', index_col='date')
```

#### **Excel文件**
-  `.read_excel()`  : 读取Excel
-  `.to_excel()`  : 写入Excel
-  **安装**  ：需要`openpyxl`库

```python
# 读取Excel（指定sheet）
df = pd.read_excel('data.xlsx', sheet_name='Sheet1')

# 写入Excel（多个sheet）
with pd.ExcelWriter('output.xlsx') as writer:
    df1.to_excel(writer, sheet_name='Data1')
    df2.to_excel(writer, sheet_name='Data2')
```

#### **HDF5格式**
- 高性能二进制格式，支持大文件
-  `.read_hdf()`  和 `.to_hdf()`
- 支持**query**和**chunked reading**

```python
# 写入HDF5
df.to_hdf('data.h5', key='df', mode='w')

# 读取HDF5（支持query）
large_df = pd.read_hdf('data.h5', key='df', where='column > 10')
```

#### **Parquet格式**
- 列式存储，压缩高效
-  `.read_parquet()`  和 `.to_parquet()`
- 支持复杂**data types**

```python
# 写入Parquet
df.to_parquet('data.parquet')

# 读取Parquet
df = pd.read_parquet('data.parquet')
```

#### **Pickle格式**
- 保存完整Python对象（包括types和functions）
- 仅用于**可信数据**（安全风险）

```python
# 写入Pickle
df.to_pickle('data.pkl')

# 读取Pickle
df = pd.read_pickle('data.pkl')
```

### Data Cleaning
#### **缺失值处理（Missing Values）**
-  `.isna()`  : 检测缺失值
-  `.dropna()`  : 删除含缺失值的行/列
-  `.fillna()`  : 填充缺失值

```python
# 检测缺失值
missing_mask = df.isna()
print(missing_mask.sum())  # 每列缺失值数量

# 删除缺失值
df_cleaned = df.dropna()  # 删除任何含缺失值的行
df_column_cleaned = df.dropna(subset=['important_col'])  # 仅删除特定列缺失的行

# 填充缺失值
df_filled = df.fillna(0)  # 用0填充
df_filled_mean = df.fillna(df.mean())  # 用均值填充

# 前向填充（ffill）
df_ffill = df.fillna(method='ffill')
```

#### **重复值处理（Duplicates）**
-  `.duplicated()`  : 检测重复行
-  `.drop_duplicates()`  : 删除重复行

```python
# 检测重复值
duplicates = df.duplicated()
print(f"重复行数: {duplicates.sum()}")

# 删除重复值（保留首次出现）
df_unique = df.drop_duplicates()

# 删除特定列的重复值
df_unique_subset = df.drop_duplicates(subset=['id'])
```

#### **异常值处理（Outliers）**
- 使用统计方法或IQR规则识别
- 替换或删除异常值

```python
# 使用IQR方法检测异常值
Q1 = df['value'].quantile(0.25)
Q3 = df['value'].quantile(0.75)
IQR = Q3 - Q1

# 异常值mask
outlier_mask = (df['value'] < Q1 - 1.5 * IQR) | (df['value'] > Q3 + 1.5 * IQR)

# 处理异常值（替换为边界值）
df['value_cleaned'] = df['value'].clip(lower=Q1 - 1.5 * IQR, upper=Q3 + 1.5 * IQR)
```

#### **数据类型转换（Type Conversion）**
-  `.astype()`  : 转换dtype
-  `.to_numeric()`  : 强制转换为数值

```python
# 转换数据类型
df['date'] = pd.to_datetime(df['date'])
df['price'] = pd.to_numeric(df['price'], errors='coerce')  # 无效值转为NaN
```

### Reshape
#### **Pivot（长→宽）**
-  `.pivot()`  : 长格式转为宽格式
-  `.pivot_table()`  : 支持聚合的pivot

```python
# 长格式数据
long_data = pd.DataFrame({
    'date': ['2020-01', '2020-01', '2020-02', '2020-02'],
    'variable': ['A', 'B', 'A', 'B'],
    'value': [10, 15, 12, 18]
})

# Pivot为宽格式
wide_data = long_data.pivot(index='date', columns='variable', values='value')
print(wide_data)

# Pivot table（带聚合）
pivot_table = long_data.pivot_table(
    index='date', columns='variable', values='value', aggfunc='mean'
)
```

#### **Melt（宽→长）**
-  `.melt()`  : 宽格式转为长格式

```python
# 宽格式数据
wide_data = pd.DataFrame({
    'date': ['2020-01', '2020-02'],
    'A': [10, 12],
    'B': [15, 18]
})

# Melt为长格式
long_data = wide_data.melt(id_vars='date', var_name='variable', value_name='value')
print(long_data)
```

#### **Stack和Unstack**
-  `.stack()`  : 将columns压缩到index（宽→长）
-  `.unstack()`  : 将index展开为columns（长→宽）

```python
# 创建MultiIndex DataFrame
df = pd.DataFrame(
    np.random.randn(4, 2),
    index=pd.MultiIndex.from_product([['A', 'B'], [1, 2]]),
    columns=['X', 'Y']
)

# Stack操作
stacked = df.stack()
print(stacked)

# Unstack操作
unstacked = stacked.unstack()
print(unstacked)

# 指定unstack level
unstacked_level0 = stacked.unstack(level=0)  # unstack第一层index
```


### Merge
#### **Concat（连接）**
- 沿行或列方向连接
-  `axis=0`  : 垂直连接（增加行）
-  `axis=1`  : 水平连接（增加列）

```python
# 垂直连接
df1 = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
df2 = pd.DataFrame({'A': [5, 6], 'B': [7, 8]})
concat_vert = pd.concat([df1, df2])  # 默认axis=0

# 水平连接
df3 = pd.DataFrame({'C': [9, 10], 'D': [11, 12]})
concat_horiz = pd.concat([df1, df3], axis=1)
```

#### **Merge（合并）**
- 基于列值匹配合并
-  `how`  : `inner`（默认）、`left`、`right`、`outer`
-  `on`  : 指定匹配的列

```python
# 创建DataFrame
df1 = pd.DataFrame({'key': ['A', 'B', 'C'], 'value1': [1, 2, 3]})
df2 = pd.DataFrame({'key': ['A', 'B', 'D'], 'value2': [4, 5, 6]})

# Inner merge（默认）
merged_inner = pd.merge(df1, df2, on='key')
print(merged_inner)  # 仅保留key为A,B的行

# Outer merge（保留所有key）
merged_outer = pd.merge(df1, df2, on='key', how='outer')
print(merged_outer)  # 包含A,B,C,D，缺失值用NaN填充

# Left merge（保留左表所有行）
merged_left = pd.merge(df1, df2, on='key', how='left')
```

#### **Join（连接）**
- 基于index的连接
- 类似merge但使用index作为key

```python
# 设置index
df1_indexed = df1.set_index('key')
df2_indexed = df2.set_index('key')

# Join操作
joined = df1_indexed.join(df2_indexed, how='inner')
print(joined)
```

### Groupby

**核心概念**：使用**groupby**进行分组计算，生成**aggregations**、**transforms**、**filters**

#### **分组聚合（Groupby Aggregation）**
-  `.groupby()`  → **split**-apply-combine模式
-  `.agg()`  : 对每个组应用聚合函数

```python
# 加载数据
url = "https://datascience.quantecon.org/assets/data/employment.csv"
emp = pd.read_csv(url)
emp = emp.set_index('year')

# 按行业和州分组，计算工资均值
industry_state_means = emp.groupby(['industry', 'state'])['earnings'].mean()
print(industry_state_means.head())

# Reset index还原为多列DataFrame
industry_state_means_df = industry_state_means.reset_index()
```

#### **多函数聚合**
- 对同一列应用多个函数
- 对不同列应用不同函数

```python
# 对一列应用多个函数
multi_agg = emp.groupby('industry')['earnings'].agg(['mean', 'std', 'count'])
print(multi_agg)

# 对不同列应用不同函数
custom_agg = emp.groupby('industry').agg({
    'earnings': 'mean',
    'age': ['min', 'max'],
    'state': 'count'
})
print(custom_agg)
```

#### **分组转换（Groupby Transform）**
- 计算组统计量并广播回原DataFrame
- 保持原DataFrame形状

```python
# 计算行业平均工资并添加到原数据
emp_copy = emp.copy()
emp_copy['industry_avg_earnings'] = (
    emp.groupby('industry')['earnings'].transform('mean')
)
print(emp_copy.head())

# 计算行业平均工资的百分比
emp_copy['earnings_vs_industry_avg'] = (
    emp_copy['earnings'] / emp_copy['industry_avg_earnings']
)
```

#### **分组过滤（Groupby Filter）**
- 根据组属性筛选组

```python
# 保留样本数大于1000的行业
large_industries = emp.groupby('industry').filter(
    lambda x: len(x) > 1000
)
print(f"保留行业数: {large_industries['industry'].nunique()}")
```

#### **分组迭代（Groupby Iteration）**
- 遍历每个组

```python
# 遍历每个行业组
for industry_name, group in emp.groupby('industry'):
    print(f"行业: {industry_name}, 样本数: {len(group)}")
    if len(group) < 1000:
        print(f"  -> 样本太少，可能需要过滤")
```

### Timeseries
#### **Resample（重采样）**
-  `.resample()`  : 改变时间序列频率
-  `how`  : 聚合方法（`mean`, `sum`, `first`, `last`）

```python
# 加载数据
url = "https://datascience.quantecon.org/assets/data/employment.csv"
emp = pd.read_csv(url, parse_dates=['date'])
emp = emp.set_index('date')[['earnings']]

# 按年重采样
yearly_means = emp.resample('Y').mean()
print(yearly_means.head())

# 按周重采样
weekly_means = emp.resample('W').mean()
print(weekly_means.head())
```

#### **Rolling（滚动窗口）**
-  `.rolling()`  : 滚动窗口计算
-  `.rolling(window).mean()`  : 移动平均

```python
# 计算30天移动平均
rolling_mean = emp['earnings'].rolling(30).mean()
print(rolling_mean.head(35))  # 前29个值为NaN（窗口不足）

# 计算20天滚动标准差
rolling_std = emp['earnings'].rolling(20).std()
print(rolling_std.head(25))

# 最小周期设置
rolling_min10 = emp['earnings'].rolling(20, min_periods=10).mean()
print(rolling_min10.head(15))  # 前10-19个值可用
```

#### **日期偏移（Date Offsets）**
-  `pd.DateOffset`  : 灵活日期加减
-  `+ pd.DateOffset(months=1)`  : 加一个月

```python
# 创建DatetimeIndex
dates = pd.date_range('2020-01-31', periods=3, freq='M')

# 加上一个月偏移
offset_dates = dates + pd.DateOffset(months=1)
print(offset_dates)

# 使用offsets移动数据
shifted = emp.shift(30)  # 向后移动30天
print(shifted.head())
```

#### **时间差（Time Deltas）**
-  `pd.Timedelta`  : 时间差对象
- 支持天数、小时、分钟等单位

```python
# 创建时间差
td = pd.Timedelta(days=7)
print(td)

# 计算时间差
time_diff = emp.index[-1] - emp.index[0]
print(f"时间跨度: {time_diff.days}天")

# 筛选特定时间范围
start_date = pd.Timestamp('2000-01-01')
end_date = start_date + pd.Timedelta(days=365)
year_data = emp.loc[start_date:end_date]
print(year_data.shape)
```

#### **时间序列绘图**
- 直接使用**DatetimeIndex**绘图，自动格式化x轴

```python
import matplotlib.pyplot as plt

# 计算原始数据和30天滚动平均
ax = emp['earnings'].plot(label='Raw', alpha=0.5)
emp['earnings'].rolling(30).mean().plot(ax=ax, label='30-day Rolling Mean')

ax.set_ylabel('Earnings')
ax.set_title('Earnings with Rolling Mean')
ax.legend()
plt.show()
```